Урок 5. Объединение таблиц. Представления. CTE

1.Написать запрос, который выведет количество уникальных персонажей, участвующих в PvP-активностях, для каждой расы персонажей из таблицы characters. Для этого нужно соединить таблицы pvp_log и characters по ID персонажа, а затем посчитать количество уникальных ID персонажей, сгруппировав данные по расам и применив агрегатную функцию.
2.В одном запросе вывести:
количество уникальных персонажей, участвующих в PvP;
количество платящих игроков, участвующих в PvP (платящими считаем игроков, которые делали покупки в игровом магазине);
3.посчитать долю платящих среди участвующих в PvP. Для этого можно выбрать уникальных персонажей из таблицы pvp_log, присоединить к ним внешним соединением данные из таблицы shop_log (например, сумму транзакций) по соответствию ID персонажа, а затем во внешнем запросе посчитать общее количество персонажей и количество значений, присоединённых из таблицы shop_log. Для расчёта доли платящих нужно поделить второе значение на первое. При этом нельзя забывать про особенности деления в PostgreSQL — для получения точного значения необходимо делить значения типа FLOAT.
4.Для каждой даты вывести количество уникальных персонажей, которые ходили в подземелья или участвовали в PvP или покупали что-либо в магазине. На каждую дату должно быть одно значение количества персонажей. Данные отсортировать по датам по возрастанию. Необходимо объединить выборки даты и charid из таблиц 5.shop_log, pvp_log и dungeon_log, а затем во внешнем запросе сгруппировать данные по датам, посчитав количество уникальных charid.
Выполнить задание из пункта 1, используя общее табличное выражение. Для этого необходимо поместить в конструкцию WITH запрос, выполняющий объединение таблиц pvp_log и characters, а затем в основном запросе выполнить группировку по расам и применение агрегатной функции.

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')



In [3]:
import pandas as pd
import matplotlib

df = pd.read_sql(f'''
SELECT c.charrace, count(*) as cnt
FROM (
    SELECT DISTINCT p.charid 
    FROM gu_game_analytics_1188.pvp_log as p
    ) as p
JOIN (
    SELECT id, charrace
    FROM gu_game_analytics_1188.characters as c
    ) as c
ON p.charid = c.id
GROUP BY c.charrace
''', con=engine)
df

,charrace,cnt
0,Orc,2381
1,Human,3717
2,Dwarf,2932
3,Elf,3735


In [4]:
df = pd.read_sql(f'''
SELECT pvp_char, payer, sum(t.payer/t.pvp_char) as pvp_payer
FROM(
    SELECT cast(count(*) as float) as pvp_char, 
    cast(count(price) as float) as payer
    FROM (
        SELECT DISTINCT p.charid
        FROM gu_game_analytics_1188.pvp_log as p
        ) as p
    LEFT JOIN (
        SELECT s.charid, sum(itemprice*itemcount) as price
        FROM gu_game_analytics_1188.shop_log as s
        GROUP BY s.charid
        ) as s
    ON p.charid = s.charid
    ) as t
GROUP BY pvp_char, payer
''', con=engine)
df

,pvp_char,payer,pvp_payer
0,12765.0,1370.0,0.107325


In [5]:
df = pd.read_sql(f'''
SELECT time, count(distinct charid)
FROM (
    SELECT cast(time as date), charid
    FROM gu_game_analytics_1188.dungeon_log
    UNION ALL
    SELECT cast(time as date), charid
    FROM gu_game_analytics_1188.pvp_log
    UNION ALL
    SELECT cast(time as date), charid
    FROM gu_game_analytics_1188.shop_log
) as t
GROUP BY time
ORDER BY time
''', con=engine)
df

,time,count
0,2018-08-01,11486
1,2018-08-02,13002
2,2018-08-03,13366
3,2018-08-04,12818
4,2018-08-05,12231
5,2018-08-06,17557
6,2018-08-07,15308
7,2018-08-08,13932
8,2018-08-09,13870
9,2018-08-10,13566


In [6]:
df = pd.read_sql(f'''
WITH p_c as (
    SELECT * 
    FROM gu_game_analytics_1188.pvp_log as p
    JOIN gu_game_analytics_1188.characters as c
    ON p.charid = c.id
)
SELECT distinct charrace, count(distinct id)
FROM p_c
GROUP BY charrace
''', con=engine)
df

,charrace,count
0,Dwarf,2932
1,Elf,3735
2,Human,3717
3,Orc,2381
